In [1]:
import pandas as pd
import numpy as np
import altair as alt
import nbformat

import json
import glob
import os

In [2]:
df = pd.read_csv('data/raw/lol/2023_LoL_esports_match_data_from_OraclesElixir.csv')
player = (df
          .assign(deaths=np.where(df.deaths == 0, 1, df.deaths))
          .assign(kd = lambda d_: (d_['kills'] + d_['assists']) / d_['deaths'])
          .groupby('playerid').agg({'kd': 'mean', 'result': 'sum', 'result': ['sum', 'count']})
          .reset_index()
          .assign(wr=lambda d_: d_['result'],['sum'] / d_['result']['count'])[['playerid', 'kd', 'wr']])

player.columns = player.columns.droplevel(1)

player = (player
          .merge(df
                 .groupby(['playerid', 'position'])['gameid']
                 .count()
                 .reset_index()
                 .drop_duplicates(), how='inner', on='playerid')
                 .rename(columns={'gameid': 'games_played'}))

/var/folders/p0/cx2wcl6d7w79c__n36_zynk00000gn/T/ipykernel_15124/186090432.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/raw/lol/2023_LoL_esports_match_data_from_OraclesElixir.csv')


In [3]:
alt.Chart(player.loc[(player['wr'] != 0) & (player['wr'] != 1)]).mark_circle().encode(
    x='kd:Q',
    y='wr:Q',
    color='position',
    size='games_played'
).properties(
    width=600,
    height=600
)

alt.Chart(...)

In [4]:
IN_DIR = './data/raw/lol'

all_files = glob.glob(os.path.join(IN_DIR, '*.csv'))
df = (pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
      .assign(ts = lambda _d: (pd.to_datetime(_d['date']).astype(int) / 10**9).astype(int)))

nba = pd.read_csv('./data/raw/nba/games.csv').assign(ts = lambda _d: (pd.to_datetime(_d['GAME_DATE_EST']).astype(int) / 10**9).astype(int))

home = df.loc[df['side'] == 'Blue', ['teamid', 'gameid', 'result', 'ts', 'gamelength', 'patch']].dropna().drop_duplicates()
away = df.loc[df['side'] == 'Red', ['teamid', 'gameid']].dropna().drop_duplicates()

matches = home.merge(away, how='inner', on='gameid')

/var/folders/p0/cx2wcl6d7w79c__n36_zynk00000gn/T/ipykernel_15124/2090102045.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = (pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
/var/folders/p0/cx2wcl6d7w79c__n36_zynk00000gn/T/ipykernel_15124/2090102045.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = (pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)


In [5]:
loleda = (matches
       .groupby('patch')
       .agg({'result': ['sum', 'count']})
       .reset_index()
       .assign(wr=lambda d_: (d_['result']['sum'] / d_['result']['count']))[['patch', 'wr']])
loleda.columns = loleda.columns.droplevel(1)

In [6]:
nbaeda = (nba
          .groupby('SEASON')
          .agg({'HOME_TEAM_WINS': ['sum', 'count']})
          .reset_index()
          .assign(wr=lambda d_: (d_['HOME_TEAM_WINS']['sum'] / d_['HOME_TEAM_WINS']['count']))
          [['SEASON', 'wr']])
nbaeda.columns = nbaeda.columns.droplevel(1)

In [9]:
alt.Chart(loleda).mark_circle(size=50).encode(
    x='patch:N',
    y='wr:Q'
)

alt.Chart(...)

In [8]:
alt.Chart(nbaeda).mark_circle(size=50).encode(
    x='SEASON:N',
    y='wr:Q'
)

alt.Chart(...)